In [ ]:
!pip uninstall python-louvain -y

Found existing installation: python-louvain 0.16
Uninstalling python-louvain-0.16:
  Successfully uninstalled python-louvain-0.16


In [ ]:
!pip install python-louvain

  Using cached python_louvain-0.16-py3-none-any.whl


# **Import Libraries**

In [ ]:
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import community as community_louvain
import matplotlib.pyplot as plt
from collections import Counter
from collections import defaultdict

# **Import DataSet**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/BA Project/productDataset.csv"
df = pd.read_csv(path)
df

,id,category_name,titles,min_price,max_price,avg_price,min_num_shops,max_num_shops,avg_num_shops
0,1867826,میکروسکوپ,['میکروسکوپ اپتیکی سلسترون مدل 44121 CGL'],1900000.0,2082000.0,1.928314e+06,1,2,1.655303
1,419611,ماشین اسباب بازی,['ماشین بازی فورد موستانگ مایستو مدل 2015 Ford...,NaN,NaN,NaN,0,0,0.000000
2,288575,کتاب و مجلات,['کتاب راهنمای کاربردی عیب یابی و تعمیر موتور ...,65000.0,100000.0,7.396341e+04,1,2,1.743902
3,4614227,لپ تاپ و نوت بوک,['لپ تاپ مایکروسافت مدل سرفیس لپتاپ با پردازند...,43887600.0,48553000.0,4.648145e+07,1,1,1.000000
4,4108604,کتاب و مجلات,"['کتاب اقیانوسی در ذهن كلر وندرپول', 'اقیانوسی...",36660.0,74000.0,3.788296e+04,1,3,2.940000
...,...,...,...,...,...,...,...,...,...
3612272,1823783,ساعت مچی عقربه‌ ای و دیجیتالی,['ساعت مچی آنالوگ مردانه سیاه برند diesel DZ45...,25618406.0,25618406.0,2.561841e+07,1,1,1.000000
3612273,6785207,کاپشن، بارانی و پالتو مردانه,['کاپشن روزمره مردانه سیاه برند columbia 14204...,3426688.0,3426688.0,3.426688e+06,1,1,1.000000
3612274,269414,کاپشن، بارانی و پالتو مردانه,['کاپشن روزمره مردانه سرمه\u200cای برند columb...,10068875.0,10068875.0,1.006888e+07,1,1,1.000000
3612275,746225,کیف و کوله,['کیف ورزشی مردانه سیاه برند puma YAGM18931'],2677313.0,2677313.0,2.677313e+06,1,1,1.000000


# **Extract relevant columns**

In [ ]:
product_data = df[['category_name', 'titles']]
product_data

,category_name,titles
0,میکروسکوپ,['میکروسکوپ اپتیکی سلسترون مدل 44121 CGL']
1,ماشین اسباب بازی,['ماشین بازی فورد موستانگ مایستو مدل 2015 Ford...
2,کتاب و مجلات,['کتاب راهنمای کاربردی عیب یابی و تعمیر موتور ...
3,لپ تاپ و نوت بوک,['لپ تاپ مایکروسافت مدل سرفیس لپتاپ با پردازند...
4,کتاب و مجلات,"['کتاب اقیانوسی در ذهن كلر وندرپول', 'اقیانوسی..."
...,...,...
3612272,ساعت مچی عقربه‌ ای و دیجیتالی,['ساعت مچی آنالوگ مردانه سیاه برند diesel DZ45...
3612273,کاپشن، بارانی و پالتو مردانه,['کاپشن روزمره مردانه سیاه برند columbia 14204...
3612274,کاپشن، بارانی و پالتو مردانه,['کاپشن روزمره مردانه سرمه\u200cای برند columb...
3612275,کیف و کوله,['کیف ورزشی مردانه سیاه برند puma YAGM18931']


In [ ]:
product_data.to_csv("/content/drive/MyDrive/BA Project/Category_Title.csv")

# **bipartite graph**

In [ ]:
B = nx.Graph()

In [ ]:
for index, row in product_data.iterrows():
    category = row['category_name']
    products = []
    products = [product.strip() for product in row['titles'].split(',')]

    # Add the category node with attribute 'bipartite=0'
    B.add_node(category, bipartite=0)

    # Add product nodes with attribute 'bipartite=1'
    for product in products:
        B.add_node(product, bipartite=1)
        B.add_edge(category, product)

# **one-mode projection for categories**

In [ ]:
category_nodes = {n for n, d in B.nodes(data=True) if d['bipartite'] == 0}

In [ ]:
categoryproduct_network = bipartite.weighted_projected_graph(B, category_nodes)

#Community Detection

In [ ]:
partition = community_louvain.best_partition(categoryproduct_network)

In [ ]:
communities = defaultdict(list)
for node, community in partition.items():
    communities[community].append(node)

# Convert defaultdict to a list of lists for easier handling
communities_list = list(communities.values())

# Print the list of communities
communities_list

[['تلمبه',
  'میکسر صدا',
  'تسمه دینام و پروانه خودرو',
  'مکمل آهن',
  'اسپیکر لپ تاپ',
  'کیلومتر موتور سیکلت و لوازم جانبی',
  'چرخ خیاطی و ریسندگی',
  'لوازم جانبی پاوربانک',
  'گلگیر موتو رسیکلت',
  'گهواره و تاب برقی',
  'مانتو، پانچ و رویه',
  'مینی کامپیوتر',
  'برد بورد',
  'جک بازویی و پارکینگی',
  'مکمل سلنیوم',
  'دوچرخه کوهستان سایز 26 و بالاتر',
  'انگشتر طلا',
  'مانیتور',
  'ساق بند ورزشی',
  'قاب و کاور موتور خودرو',
  'کابل و تبدیل USB',
  'گوشواره طلا',
  'لوازم صابون سازی',
  'باتری موتور سیکلت',
  'کیف و کاور عینک',
  'فرش نمدی',
  'همزن دستی',
  'لوازم گلسازی',
  'فشارسنج پزشکی',
  'کابل و سیم تلفن',
  'عطاری و گیاهان دارویی',
  'کرم و ژل لاغری',
  'مانیتور آیفون تصویری',
  'سرم ویتامینه پوست',
  'جاباتری و شارژر باتری',
  'کیف و کاور هدفون و هندزفری',
  'اون توستر',
  'سنگ یشم',
  'بند انداز برقی',
  'محافظ و نگهدارنده کابل',
  'کیت آزمایشگاهی',
  'سایر تیشرت خردسال و نوجوان',
  'مراقبت چشم و گوش سگ و گربه',
  'سایر الکتروموتور و لوازم جانبی',
  'مهر و استامپ',


In [ ]:
communities_list

AttributeError: 'list' object has no attribute 'info'